# Exercises on Code-Based Cryptography

In [ ]:
from sage.all import matrix, GF, identity_matrix, vector
import cme, bike, isd
import matplotlib.pyplot as plt
import time

### Exercise 1. Evaluate the performance of the Classic McEleice Cryptosystem for different parameters

**Parameters**
- **n:** Length of the code.
- **t:** Error correction capability.
- **m:** Degree of the finite field of the code.
- **tn:** Number of trials per parameter set.

In [ ]:
# Define the parameters for the test batch
batch = [((7, 1, 3), [], [], [], [], []),
         ((15, 3, 4), [], [], [], [], []),
         ((31, 6, 5), [], [], [], [], []),
         ((63, 10, 6), [], [], [], [], []),
         ((127, 17, 7), [], [], [], [], []),
         ((196, 6, 8), [], [], [], [], []),
         ((256, 7, 8), [], [], [], [], []),
         ((490, 12, 9), [], [], [], [], []),
         ((3488, 64, 12), [], [], [], [], [])]
trials = 5

In [ ]:
# Perform the evaluations
for (p, l, ml, kl, tl, mtl) in batch:
    
    # Encryption trials
    for _ in range(trials):
        
        # Encapsulate the Classic McEleice parameters
        params = cme.classic_mceleice_encapsulate_parameters(p[0], p[1], p[2])
        
        # Start timing for key generation
        cme_key_start = time.process_time() * 1000
        
        # Generate the key pair
        key_pair = cme.classic_mceleice_generate_key_pair(params)
        
        # End timing for key generation
        cme_key_end = time.process_time() * 1000
        
        # Generate the error vector
        e = cme.classic_mceleice_generate_error_vector(params)
        
        # Generate the capsule and session key
        c, K = cme.classic_mceleice_encapsulate(key_pair[1], e)
        
        # Start timing for Classic McEleice decapsulation
        cme_start = time.process_time() * 1000
        
        # Decrypt the secret vector with private key
        nK = cme.classic_mceleice_decapsulate(params, key_pair[0], c)
        
        # End timing for Classic McEleice decapsulation
        cme_end = time.process_time() * 1000
        
        # Start timing for ISD attack
        isd_start = time.process_time() * 1000
        
        # Calculate the public key
        I = matrix(GF(2), identity_matrix(key_pair[1].nrows()))
        H = I.augment(key_pair[1])
        
        # Perform the attack
        a = isd.isd_prange(H, c[0], params[1])
        
        # Generate a session key using the attack
        if a is not None:
            aK = cme.classic_mceleice_decapsulate(params, key_pair[0], c, a)
        else:
            aK = None
        
        # End timing for ISD attack
        isd_end = time.process_time() * 1000
        
        # Save the results for statistical purposes
        l.append(K == nK)
        ml.append(K == aK)
        kl.append(cme_key_end - cme_key_start)
        tl.append(cme_end - cme_start)
        mtl.append(isd_end - isd_start)

In [ ]:
# Calculate the axis arrays
x = [f"(n:{p[0]},t:{p[1]},m:{p[2]},k:{p[0] - p[1] * p[2]})" for (p, _, _, _, _, _) in batch]
y_1 = [m*t*(n - m*t) + n * (2**m) for ((n, t, m), _, _, _, _, _) in batch]
y_2 = [(sum(l) / len(l)) * 100 for (_, l, _, _, _, _) in batch]
y_3 = [(sum(kl) / len(kl)) for (_, _, _, kl, _, _) in batch]
y_4 = [(sum(tl) / len(tl)) for (_, _, _, _, tl, _) in batch]

# Define the plot to show key sizes depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_1, marker='.')
plt.title("Key Size (in Bytes) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Key Size (in Bytes)")
plt.grid(True)
plt.show()

# Define the plot to show decryption sucess rate depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_2, marker='.')
plt.title("Decryption Sucess Rate per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Decryption Sucess Rate")
plt.ylim(0, 105)
plt.grid(True)
plt.show()

# Define the plot to show key generation time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_3, marker='.')
plt.title("Key Generation Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Key Generation Time (in ms)")
plt.grid(True)
plt.show()

# Define the plot to show decryption time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_4, marker='.')
plt.title("Decryption Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Decryption Time (in ms)")
plt.grid(True)
plt.show()

### Exercise 2. Evaluate the performance of the BIKE Cryptosystem for different parameters

**Parameters**
- **r:** Length of the code.
- **w:** Key hamming weight.
- **t:** Error hamming weight.
- **l:** Key and message length in bits.
- **tn:** Number of trials per parameter set.

In [ ]:
# Define the parameters for the test batch
batch = [((293, 18, 15, 128), [], [], [], [], []),
         ((587, 42, 19, 128), [], [], [], [], []),
         ((12323, 142, 134, 128), [], [], [], [], []),
         ((24659, 206, 199, 192), [], [], [], [], []),
         ((40973, 274, 264, 256), [], [], [], [], [])]
trials = 10

In [ ]:
# Perform the evaluations
for (p, l, ml, kl, tl, mtl) in batch:
    
    # Encryption trials
    for _ in range(trials):
        
        # Encapsulate the BIKE parameters
        params = bike.bike_encapsulate_parameters(p[0], p[1], p[2], p[3])
        
        # Start timing for key generation
        bike_key_start = time.process_time() * 1000
        
        # Generate the key pair
        key_pair = bike.bike_generate_key_pair(params)
        
        # End timing for key generation
        bike_key_end = time.process_time() * 1000
        
        # Generate the capsule and session key
        K, c = bike.bike_encapsulate(params, key_pair[1])
        
        # Start timing for Classic McEleice decapsulation
        bike_start = time.process_time() * 1000
        
        # Decrypt the secret vector with private key
        nK = bike.bike_decapsulate(params, key_pair[0], c)
        
        # End timing for Classic McEleice decapsulation
        bike_end = time.process_time() * 1000
        
        # Start timing for ISD attack
        isd_start = time.process_time() * 1000
        
        # Check if the attack is feasable
        if (params[0] < 12323):
        
            # Calculate the public key
            Hc = matrix.circulant(vector(GF(2), key_pair[1]))
            I = matrix.identity(GF(2), params[0])
            H = I.augment(Hc)
            
            # Perform the attack
            a = isd.isd_prange(H, vector(GF(2), c[0]), params[1])
            
        else:
            a = None
        
        # Generate a session key using the attack
        if a is not None:
            
            # Create the error tuple
            ae = (list(a[0:params[0]]), list(a[params[0]:2*params[0]]))
            
            # Execute the attack
            aK = bike.bike_decapsulate(params, key_pair[0], c, e = ae)
            
        else:
            aK = None
        
        # End timing for ISD attack
        isd_end = time.process_time() * 1000
        
        # Save the results for statistical purposes
        l.append(K == nK)
        ml.append(K == aK)
        kl.append(bike_key_end - bike_key_start)
        tl.append(bike_end - bike_start)
        mtl.append(isd_end - isd_start)

In [ ]:
# Calculate the axis arrays
x = [f"(r:{p[0]},w:{p[1]},t:{p[2]},l:{p[3]})" for (p, _, _, _, _, _) in batch]
y_1 = [3*r + (l // 8)  for ((r, w, t, l), _, _, _, _, _) in batch]
y_2 = [(sum(l) / len(l)) * 100 for (_, l, _, _, _, _) in batch]
y_3 = [(sum(kl) / len(kl)) for (_, _, _, kl, _, _) in batch]
y_4 = [(sum(tl) / len(tl)) for (_, _, _, _, tl, _) in batch]

# Define the plot to show key sizes depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_1, marker='.')
plt.title("Key Size (in Bytes) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Key Size (in Bytes)")
plt.grid(True)
plt.show()

# Define the plot to show decryption sucess rate depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_2, marker='.')
plt.title("Decryption Sucess Rate per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Decryption Sucess Rate")
plt.ylim(0, 105)
plt.grid(True)
plt.show()

# Define the plot to show key generation time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_3, marker='.')
plt.title("Key Generation Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Key Generation Time (in ms)")
plt.grid(True)
plt.show()

# Define the plot to show decryption time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_4, marker='.')
plt.title("Decryption Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Decryption Time (in ms)")
plt.grid(True)
plt.show()

### Exercise 3. Evaluate the performance of ISD attack for both Cryptosystems and find an accurate parameter threshold

In [ ]:
# Calculate the axis arrays
x = [f"(n:{p[0]},t:{p[1]},m:{p[2]},k:{p[0] - p[1] * p[2]})" for (p, _, _, _, _, _) in batch]
y_1 = [(sum(l) / len(l)) * 100 for (_, _, l, _, _, _) in batch]
y_2 = [(sum(tl) / len(tl)) * 100 for (_, _, _, _, _, tl) in batch]

# Define the plot to show ISD attack sucess rate depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_1, marker='.')
plt.title("ISD Attack Sucess Rate per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("ISD Attack Sucess Rate")
plt.ylim(0, 105)
plt.grid(True)
plt.show()

# Define the plot to show attack time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_2, marker='.')
plt.title("Attack Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Attack Time (in ms)")
plt.grid(True)
plt.show()

In [ ]:
# Calculate the axis arrays
x = [f"(r:{p[0]},w:{p[1]},t:{p[2]},l:{p[3]})" for (p, _, _, _, _, _) in batch]
y_1 = [(sum(l) / len(l)) * 100 for (_, _, l, _, _, _) in batch]
y_2 = [(sum(tl) / len(tl)) * 100 for (_, _, _, _, _, tl) in batch]

# Define the plot to show ISD attack sucess rate depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_1, marker='.')
plt.title("ISD Attack Sucess Rate per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("ISD Attack Sucess Rate")
plt.ylim(0, 105)
plt.grid(True)
plt.show()

# Define the plot to show attack time depending on Parameter Set
plt.figure(figsize=(24,10))
plt.plot(x, y_2, marker='.')
plt.title("Attack Time (in ms) per Parameter Set")
plt.xlabel("Parameter Set")
plt.ylabel("Attack Time (in ms)")
plt.grid(True)
plt.show()